In [1]:
import os
import requests
from datetime import datetime
from google.cloud import storage
from google.oauth2 import service_account


BASE_URL = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv"
LIMIT = 50000

def parse_data(data_str: str) -> datetime:
    """Converte data dd/mm/aaaa para datetime"""
    return datetime.strptime(data_str, "%d/%m/%Y")


def gerar_intervalos_mensais(data_inicio: datetime, data_fim: datetime):
    """Gera pares (inicio_mes, fim_mes)"""
    intervalos = []
    atual = datetime(data_inicio.year, data_inicio.month, 1)

    while atual <= data_fim:
        if atual.month == 12:
            prox = datetime(atual.year + 1, 1, 1)
        else:
            prox = datetime(atual.year, atual.month + 1, 1)

        inicio = max(atual, data_inicio)
        fim = min(prox, data_fim)

        intervalos.append((inicio, fim))
        atual = prox

    return intervalos


def arquivo_existe(bucket, blob_name: str) -> bool:
    
    return bucket.blob(blob_name).exists()


def baixar_e_enviar_mes(bucket, inicio: datetime, fim: datetime):
    
    year = inicio.year
    month = inicio.month
    offset = 0
    total_rows = 0

    print(f"\n🔄 Processando {year}-{month:02d}")

    where = (
        f'created_date >= "{inicio.isoformat()}" AND '
        f'created_date < "{fim.isoformat()}"'
    )

    while True:
        gcs_blob_name = (
            f'landing/nyc_311/'
            f'nyc_311_{year}_{month:02d}_offset_{offset}.csv'
        )

        # 👉 PULA se já existir (retomada automática)
        if arquivo_existe(bucket, gcs_blob_name):
            print(f'⏭️  Já existe | offset {offset}')
            offset += LIMIT
            continue

        params = {
            "$where": where,
            "$limit": LIMIT,
            "$offset": offset
        }

        resp = requests.get(BASE_URL, params=params, timeout=120)

        if resp.status_code != 200:
            print(f'❌ Erro HTTP {resp.status_code} | offset {offset}')
            break

        content = resp.text
        linhas = content.strip().split("\n")

        if len(linhas) <= 1:
            break

        num_rows = len(linhas) - 1
        filename = f'nyc_311_{year}_{month:02d}_offset_{offset}.csv'

        with open(filename, "w", encoding="utf-8") as f:
            f.write(content)

        bucket.blob(gcs_blob_name).upload_from_filename(filename)
        os.remove(filename)

        print(f'✔ Enviado {num_rows} registros | offset {offset}')

        total_rows += num_rows
        offset += LIMIT

    print(f'✅ {year}-{month:02d} finalizado com {total_rows} registros')


def run_ingestion(
    data_inicio: str,
    data_fim: str,
    service_account_path: str,
    bucket_name: str
):
    # Datas
    inicio = parse_data(data_inicio)
    fim = parse_data(data_fim)

    # Credenciais
    credentials = service_account.Credentials.from_service_account_file(
        service_account_path
    )
    client = storage.Client(credentials=credentials)
    bucket = client.bucket(bucket_name)

    # Intervalos mensais
    intervalos = gerar_intervalos_mensais(inicio, fim)

    for inicio_mes, fim_mes in intervalos:
        baixar_e_enviar_mes(bucket, inicio_mes, fim_mes)


# =============================
# EXECUÇÃO
# =============================
if __name__ == "__main__":
    run_ingestion(
        data_inicio="01/04/2025",
        data_fim="16/12/2025",
        service_account_path=r"C:\Users\izabela.pereira\Documents\projetos\projeto_nyc\service_account.json",
        bucket_name="projeto_nyc"
    )





🔄 Processando 2025-04
⏭️  Já existe | offset 0
✔ Enviado 148104 registros | offset 50000
✔ Enviado 148254 registros | offset 100000
✔ Enviado 148200 registros | offset 150000
✔ Enviado 148290 registros | offset 200000
✔ Enviado 67184 registros | offset 250000
✅ 2025-04 finalizado com 660032 registros

🔄 Processando 2025-05
✔ Enviado 148070 registros | offset 0
✔ Enviado 148418 registros | offset 50000
✔ Enviado 148202 registros | offset 100000
✔ Enviado 147752 registros | offset 150000
✔ Enviado 148326 registros | offset 200000
✔ Enviado 134184 registros | offset 250000
✅ 2025-05 finalizado com 874952 registros

🔄 Processando 2025-06
✔ Enviado 148010 registros | offset 0
✔ Enviado 148098 registros | offset 50000
✔ Enviado 147820 registros | offset 100000
✔ Enviado 147968 registros | offset 150000
✔ Enviado 148406 registros | offset 200000
✔ Enviado 91705 registros | offset 250000
✅ 2025-06 finalizado com 832007 registros

🔄 Processando 2025-07
✔ Enviado 148062 registros | offset 0
✔ E